## 79. Manejo de nombres de columnas duplicados:

In [1]:
%load_ext autoreload
%autoreload 2

import os
path = os.environ.get("HOME")
os.chdir(f"{path}/github/pyspark-hand-on")
from src.utils.utils import spark_session
spark = spark_session()

25/10/30 23:50:36 WARN Utils: Your hostname, personal-computer resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/10/30 23:50:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/30 23:50:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark.sql.functions import col
workers = spark.read.parquet("./data/empleados.parquet")
# workers = workers.withColumn("num_dpto", col("id"))
workers.show()

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Pedro|       0|
| Laura|      34|
|Sandro|      31|
+------+--------+



In [11]:
departments = spark.read.parquet("./data/departamentos.parquet")
departments = departments.withColumn("num_dpto", col("id"))
departments.show()

+---+-----------+--------+
| id|nombre_dpto|num_dpto|
+---+-----------+--------+
| 31|     letras|      31|
| 33|    derecho|      33|
| 34| matemática|      34|
| 35|informática|      35|
+---+-----------+--------+



In [17]:
df_dupl = workers.join(departments, workers["num_dpto"]==departments["num_dpto"])
df_dupl.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- num_dpto: long (nullable = true)
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)
 |-- num_dpto: long (nullable = true)



In [21]:
# Para evitar ambiguedad, es posible usar df origen:
df_dupl.select(workers["num_dpto"]).show()

+--------+
|num_dpto|
+--------+
|      33|
|      33|
|      34|
|      34|
|      31|
+--------+



In [22]:
df2 = workers.join(departments, "num_dpto")
df2.printSchema()

root
 |-- num_dpto: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)



In [24]:
df3 = workers.join(departments, ["num_dpto"])
df3.show()

+--------+------+---+-----------+
|num_dpto|nombre| id|nombre_dpto|
+--------+------+---+-----------+
|      31|Sandro| 31|     letras|
|      33| Katia| 33|    derecho|
|      33|  Luis| 33|    derecho|
|      34| Laura| 34| matemática|
|      34|  Raul| 34| matemática|
+--------+------+---+-----------+

